# 第8回: 周期関数，直交関数系，内積空間

## 概要
第8回の講義では，微分方程式の開放としてのフーリエ変換・ラプラス変換の理解のために，周期関数，直交関数系，内積空間を学ぶ．ここでは，フーリエ級数展開の式の復習から周期関数をおさらいし，関数の近似を直交関数の重ね合わせで表現する方法を学ぶ．そして，最後に直交関数近似を理解するために関数の内積について定義する．具体的には以下の項目の理解が目的となる．
- 周期関数の定義と性質を復習する．
- 近似表現として直交関数系による関数近似を学ぶ．
- 関数の内積を定義する．

## フーリエ級数展開
第一回目の講義でも簡単に紹介したが初めに **フーリエ級数展開** を紹介する．フーリエ級数展開は関数 $f(x)$ を $\sin$関数と$\cos$関数の足し合わせで近似する方法であり，以下で与えられる．

$$
\begin{align}
f(x) \sim  a_0 + \sum_{n=1}^{\infty} (a_n \cos nx + b_n \sin nx)
\end{align}
$$

ここで，$a_0, a_n, b_n$は以下で与えられる．

$$
\begin{align}
a_{0}&=\frac{1}{2 \pi} \int_{-\pi}^{\pi} f(x) dx \\
a_{n}&=\frac{1}{\pi} \int_{-\pi}^{\pi} f(x) \cos nx dx \quad(n=1,2,3 \cdots) \\
b_{n}&=\frac{1}{\pi} \int_{-\pi}^{\pi} f(x) \sin n x d x \quad(n=1,2,3 \cdots)
\end{align}
$$

複雑な式が現れたが，「関数 $f(x)$ を $\sin$関数と$\cos$関数の足し合わせ」で表現することを確認されたい．係数$a_0, a_n, b_n$はひとまず無視して，一番上の式を展開してみると，$\sin$波と$\cos$波の足し算であることがわかる．

$$
f(x) \sim a_0 + a_1 \cos x + b_1 \sin x + a_2 \cos 2x + b_2 \sin 2x + \cdots
$$

フーリエ解析（$\fallingdotseq$フーリエ級数展開）のポイントは複雑な関数を単純な関数（$\sin$波と$\cos$波）に分解することで複雑な関数の特性を知ることができること，また，単純な関数の組み合わせで複雑な関数を合成することにある．本講義ではこの単純な関数の足し合わせについて深掘りしていく．

## 周期関数
これもまた復習となるが上記のフーリエ級数展開で登場した$\sin$関数と$\cos$関数は **正弦波** とも呼ばれ，この正弦波は同じ波が一定間隔で繰り返される **周期関数** である．このとき波が繰り返される一定間隔のことを **周期** と呼ぶ．周期を正の定数 $T$ として，周期関数 $f$ は

$$
f(x+T) = f(x)
$$

という性質を持つ．$\sin$関数と$\cos$関数もまた周期 $T=2\pi$ の周期関数であり，三角関数の性質から，

$$
\sin(x+2\pi) = \sin(x), \quad \cos(x+2\pi) = \cos(x)
$$

となるのは自明である．$\sin$関数と$\cos$関数の周期 $T=2\pi$ のようにこの周期関数の性質を満たす最小の正の定数のことを **基本周期** と言う．$2T, 3T, ...$も周期関数の性質を満たすので周期と言えるが，これらの違いについて注意されたい．フーリエ解析ではこの周期関数による関数の近似が重要なポイントとなる．

### Pythonによる実装
では，$\sin$関数と$\cos$関数の周期関数としての性質をsympyで確認し，プロットしてみよう．

In [ ]:
from sympy import symbols, sin, cos, pi
from sympy.plotting import plot

x = symbols('x')
T = symbols('T')

f_sin = sin(x + T)
f_cos = cos(x + T)
plot(f_sin.subs(T, 2*pi), f_cos.subs(T, 2*pi), (x, -3*pi, 3*pi))

## 最小二乗法
続いて，フーリエ級数展開における周期関数の足し合わせによる関数の近似を理解するために **最小二乗法** を紹介する．最小二乗法では，データを与えられそのデータをよく表現する関数を求める **関数近似** を行う．関数近似によってデータの背後に潜む関係性を関数として捉えることができれば，データ間の対応関係や未知のデータに対する予測が可能となる．

例えば，以下のように何らかのデータを取得したとしよう．プログラムとしては，$y=2x+1$の関係が背後に潜むノイズの乗ったデータを擬似的に生成している．matplotlibを使って生成したデータ（青色の点）とその背後に潜む関係性（赤色の直線）をプロットしている．

In [ ]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt

N = 30
x_data = np.linspace(0, 1, N) # xの値
y_gt = 2 * x_data + 1 # y=2x+1の真の値
y_data = 2 * x_data + 1 + 0.1 * np.random.randn(len(x_data)) # 一定のノイズがのった観測データ

plt.scatter(x_data, y_data)
plt.plot(x_data, y_gt, c='r')

関数近似の目的は，$x$ を入力値，$y$ を $x$ に対応する目標値としたとき，$N$ 個のデータ $\{(x_i,y_i)\}_{i=1}^N$ からデータの背後に潜む赤色の関係性を求めることである．しかしながら，利用できるデータはここでは `x_data` と `y_data` のみであり，関係性に関する情報（関数の形など）は未知である．

そこで最小二乗法では，$N$ 個のデータ $\{(x_i,y_i)\}_{i=1}^N$ を与えられ，$n$個のの関数の値 $\phi_{k}(x)$ とその係数 $c_k$ との定数倍の和で表現される関数を考える（$k=1,...,n$）

$$
y_{\alpha} \approx \sum_{k=1}^{n} c_{k} \phi_{k}\left(x_{\alpha}\right), \alpha=1, \ldots, N
$$

この関数が成立するような係数 $c_1,...,c_n$ を求めることがゴールであり，これらの係数は以下の最小化問題から求めることができる．

$$
J=\frac{1}{2} \sum_{\alpha=1}^{N}\left(y_{\alpha}-\sum_{k=1}^{n} c_{k} \phi_{k}\left(x_{\alpha}\right)\right)^{2} \rightarrow \min
$$

```{admonition} 線型結合
前述の定数倍の和で表される第一式は係数 $c$ と関数 $\phi$ の値の足し合わせ（**線型結合**）になっていることが以下のように展開するとわかる．

$$
\begin{align}
y_{\alpha} &\approx \sum_{k=1}^{n} c_{k} \phi_{k}\left(x_{\alpha}\right) \\
&=c_{1} \phi_{1}\left(x_{\alpha}\right) + c_{2} \phi_{2}\left(x_{\alpha}\right) + \cdots
\end{align}
$$

上記の例のように直線で近似することを考えると，$\{\phi_1, \phi_2, c_1, c_2\}=\{1,x,b,a\}$ とすれば，上記の式は，$y_{\alpha} \approx ax_{\alpha}+b$ となり，直線の方程式 $y=ax+b$ の係数 $a,b$ をデータから求める問題になっていることがわかる．さらに，$\{\phi_1, \phi_2, \phi_3, c_1, c_2, c_3\}=\{1,x,x^2, c, b,a\}$ とすれば，二次関数 $y_{\alpha} \approx ax_{\alpha}^{2}+bx_{\alpha} + c$ で関数を近似することとなる．

ここまで考えると，$\{\phi_1, \phi_2, \phi_3\}=\{1,\cos{x},\sin{x}\}, \{c_1, c_2, c_3\}=\{a_0, a_1, b_1\}$ とすれば，$y_{\alpha} \approx a_0 + a_1 \cos{x_{\alpha}} + b_1 \sin{x_{\alpha}}$ となり，最小二乗法からフーリエ級数展開の式が書けそうなのがイメージできるかと思う．
```

では，係数 $c_k$ を求める最小化問題を深掘りする．二つ目のシグマを展開してみると，

$$
\begin{align}
J&=\frac{1}{2} \sum_{\alpha=1}^{N}\left(y_{\alpha}-\left ( c_{1} \phi_{1}\left(x_{\alpha}\right) + c_{2} \phi_{2}\left(x_{\alpha}\right) + \cdots \right )\right)^{2}
\end{align}
$$

となり，係数 $c$ をもつ関数 $\phi$ の線形結合の式と $y_{\alpha}$￼との差の計算をしていることがわかる．そして残ったシグマも展開してみると，

$$
\begin{align}
J= &\frac{1}{2} \left ( \left(y_{1}-\left ( c_{1} \phi_{1}\left(x_{1}\right) + c_{2} \phi_{2}\left(x_{1}\right) + \cdots \right )\right)^{2} \right. \\ 
& \left. + \left(y_{2}-\left ( c_{1} \phi_{1}\left(x_{2}\right) + c_{2} \phi_{2}\left(x_{2}\right) + \cdots \right )\right)^{2} + \cdots \right )
\end{align}
$$

となり，すべてのデータの差を足していることがわかる．つまりはデータを表現する関数 $\phi$ は固定で $c$ の値を変化させて関数と実際に観測されたデータの値との差（=$J$）をできるだけ小さくする問題を考えているのである．そして，$J$を小さくする$c$を求める方法はこれまで最小値の問題を解いてきた時と同じように関数の傾きを考え，傾きが$0$となる点を求めれば良い．ただし，今回は $J$ は多変数関数なので偏微分を考える必要がある．

$J$ を $c_i$ で偏微分したときの偏導関数を求める（慣れるまではシグマを展開して偏微分すると良い）．

$$
\frac{\partial J}{\partial c_i}= \sum_{\alpha=1}^{N}\left(y_{\alpha}-\sum_{k=1}^{n} c_{k} \phi_{k}\left(x_{\alpha}\right)\right)\left ( - \phi_{i} \left(x_{\alpha}\right) \right )
$$

そして，全ての係数 $c_1,...,c_n$ の偏導関数について以下の方程式を解く．

$$
\frac{\partial J}{\partial c_1}=0, \cdots, \frac{\partial J}{\partial c_n}=0
$$

この連立方程式を解くために **正規方程式** と呼ばれる行列で表現される形に書き換える．まず求めたい $c$ について偏導関数を整理する．

$$
\begin{align}
\frac{\partial J}{\partial c_i} &= \sum_{\alpha=1}^{N}\left(y_{\alpha}-\sum_{k=1}^{n} c_{k} \phi_{k}\left(x_{\alpha}\right)\right)\left ( - \phi_{i} \left(x_{\alpha}\right) \right ) \\
&=- \sum_{\alpha=1}^{N} y_{\alpha} \phi_{i} \left(x_{\alpha}\right) + \sum_{k=1}^{n} \left ( \sum_{\alpha=1}^{N}  \phi_{k}\left(x_{\alpha}\right) \phi_{i} \left(x_{\alpha}\right) \right ) c_{k}
\end{align}
$$

ここで，$ \partial J / \partial c_i = 0 $ より，

$$
\sum_{k=1}^{n} \left ( \sum_{\alpha=1}^{N}  \phi_{k}\left(x_{\alpha}\right) \phi_{i} \left(x_{\alpha}\right) \right ) c_{k} = \sum_{\alpha=1}^{N} y_{\alpha} \phi_{i} \left(x_{\alpha}\right)
$$

となる．これをすべての係数で並べてみると，

$$
\left\{\begin{matrix}
\sum_{k=1}^{n} \left ( \sum_{\alpha=1}^{N}  \phi_{k}\left(x_{\alpha}\right) \phi_{1} \left(x_{\alpha}\right) \right ) c_{k} = \sum_{\alpha=1}^{N} y_{\alpha} \phi_{1} \left(x_{\alpha}\right)\\ 
\sum_{k=1}^{n} \left ( \sum_{\alpha=1}^{N}  \phi_{k}\left(x_{\alpha}\right) \phi_{2} \left(x_{\alpha}\right) \right ) c_{k} = \sum_{\alpha=1}^{N} y_{\alpha} \phi_{2} \left(x_{\alpha}\right)\\ 
\vdots \\
\sum_{k=1}^{n} \left ( \sum_{\alpha=1}^{N}  \phi_{k}\left(x_{\alpha}\right) \phi_{n} \left(x_{\alpha}\right) \right ) c_{k} = \sum_{\alpha=1}^{N} y_{\alpha} \phi_{n} \left(x_{\alpha}\right)\\ 
\end{matrix}\right.
$$

となり，以下のように行列の形で書き直すことができる．

$$
\begin{pmatrix}
\sum_{\alpha=1}^{N} \phi_{1}\left(x_{\alpha}\right) \phi_{1}\left(x_{\alpha}\right) & \sum_{\alpha=1}^{N} \phi_{1}\left(x_{\alpha}\right) \phi_{2}\left(x_{\alpha}\right) & \cdots & \sum_{\alpha=1}^{N} \phi_{1}\left(x_{\alpha}\right) \phi_{n}\left(x_{\alpha}\right) \\
\sum_{\alpha=1}^{N} \phi_{2}\left(x_{\alpha}\right) \phi_{1}\left(x_{\alpha}\right) & \sum_{\alpha=1}^{N} \phi_{2}\left(x_{\alpha}\right) \phi_{2}\left(x_{\alpha}\right) & \cdots & \sum_{\alpha=1}^{N} \phi_{2}\left(x_{\alpha}\right) \phi_{n}\left(x_{\alpha}\right) \\
\vdots & \vdots & \ddots & \vdots \\
\sum_{\alpha=1}^{N} \phi_{n}\left(x_{\alpha}\right) \phi_{1}\left(x_{\alpha}\right) & \sum_{\alpha=1}^{N} \phi_{n}\left(x_{\alpha}\right) \phi_{2}\left(x_{\alpha}\right) & \cdots & \sum_{\alpha=1}^{N} \phi_{n}\left(x_{\alpha}\right) \phi_{n}\left(x_{\alpha}\right)
\end{pmatrix}
\begin{pmatrix}
c_1\\ 
c_2\\ 
\vdots\\ 
c_n
\end{pmatrix}
=
\begin{pmatrix}
\sum_{\alpha=1}^{N} \phi_{1}\left(x_{\alpha}\right) y_{\alpha} \\
\sum_{\alpha=1}^{N} \phi_{2}\left(x_{\alpha}\right) y_{\alpha} \\
\vdots \\
\sum_{\alpha=1}^{N} \phi_{n}\left(x_{\alpha}\right) y_{\alpha}
\end{pmatrix}
$$

この方程式を解けば係数 $c$ を求めることができ，最小二乗法による関数近似を実現できる．

```{admonition} 行列への変換
2つ以上のシグマを含む偏導関数を行列の形式へ変換するのはやや慣れが必要かもしれない．初めのうちは以下のようにシグマを展開して考えてみると良い（$c_1$の偏導関数の展開）．

$$

c_{1} \sum_{\alpha=1}^{N} \phi_{1}\left(x_{\alpha}\right) \phi_{1} \left(x_{\alpha}\right) +
c_{2} \sum_{\alpha=1}^{N} \phi_{2}\left(x_{\alpha}\right) \phi_{1} \left(x_{\alpha}\right) +
\cdots +
c_{n} \sum_{\alpha=1}^{N} \phi_{n}\left(x_{\alpha}\right) \phi_{1} \left(x_{\alpha}\right)

= \sum_{\alpha=1}^{N} y_{\alpha} \phi_{1} \left(x_{\alpha}\right)

$$

正規方程式の行列計算で偏導関数が表現できていることが確認できると思う．
```

### Pythonによる実装
では，先ほど作成した `x_data` と `y_data` から最小二乗法を実装してみる．今回は以下の線型結合で表される式，

$$
y_{\alpha} \approx \sum_{k=1}^{n} c_{k} \phi_{k}\left(x_{\alpha}\right), \alpha=1, \ldots, N
$$

について，前述したように，$\{\phi_1, \phi_2, c_1, c_2\}=\{1,x,b,a\}$ として一次関数

$$y_{\alpha} \approx ax_{\alpha}+b$$

を考える．このとき $N$ はデータ数．つまり `x_data.shape[0]`としたときの値である．まずは $N$ を確認する．

In [ ]:
x_data.shape[0]

`x_data` を作成したときの `N` と一致していることを確認されたい．続いて，式をsympyで定義していく．

In [ ]:
a = symbols('a')
b = symbols('b')

x = symbols('x')
y = symbols('y') # y_data が格納される正解用の y
y_ = symbols('y_') # 近似後の y_alpha が格納される y_

y_ = a * x + b # 関数を定義
y_

続いて，$J$ を定義する．numpy表記のように書けるのでプログラムは簡単に書ける

In [ ]:
J = (y - (a * x + b)) ** 2 / 2
J

今回，係数は $a$と$b$である．これらのパラメータに対する偏導関数$\frac{\partial J}{\partial a}, \frac{\partial J}{\partial b}$を定義する．

In [ ]:
from sympy import diff
J_a = diff(J, a)
J_a

In [ ]:
J_b = diff(J, b)
J_b

得られた偏導関数について，各データを代入して方程式を得る．具体的には次の式の計算を各係数$a,b$について行っている．

$$
\sum_{\alpha=1}^{N} y_{\alpha} \phi_{i} \left(x_{\alpha}\right) + \sum_{k=1}^{n} \left ( \sum_{\alpha=1}^{N}  \phi_{k}\left(x_{\alpha}\right) \phi_{i} \left(x_{\alpha}\right) \right ) c_{k}
$$

In [ ]:
sum_a = sum([J_a.subs([(x, _x), (y, _y)]) for _x, _y in zip(x_data, y_data)])
sum_a

In [ ]:
sum_b = sum([J_b.subs([(x, _x), (y, _y)]) for _x, _y in zip(x_data, y_data)])
sum_b

得られた連立方程式を `solve` を使って解いて係数$a,b$を求める．行っている計算は正規方程式を解いているのと同じである．

In [ ]:
from sympy import solve
coef = solve([sum_a, sum_b], [a, b])
coef

得られた係数$a,b$が実際の値に近いことがわかる．実際にプロットもしてみる

In [ ]:
y_pred = coef[a] * x_data + coef[b] 

plt.scatter(x_data, y_data)
plt.plot(x_data, y_gt, c='r')
plt.plot(x_data, y_pred, c='g')

## 直交関数系
正規方程式からパラメータ $c_k$ を求めることができたが，関数 $\phi$ を自身以外との積が $0$ となるように関数の集合 $\{ \phi_{i}(x) \}^{n}_{i=1}$ を設計するとさらに計算が簡単になる．この自身以外との積が $0$ となるような性質を **直交性** と言い，以下で定義される．

$$
\sum_{\alpha=1}^{N} \phi_{i}\left(x_{\alpha}\right) \phi_{j}\left(x_{\alpha}\right) = 0, \qquad i \neq j
$$

そして，直交性を持つ関数の集合を **直交関数系** と言う．

直交関数系で関数 $\phi$ を設計したときの正規方程式を考えると，自身以外との積が $0$ となるので正規方程式の非対角成分が $0$ となるのは明らかである．

$$
\begin{pmatrix}
\sum_{\alpha=1}^{N} \phi_{1}\left(x_{\alpha}\right) \phi_{1}\left(x_{\alpha}\right) & & & \\
& \sum_{\alpha=1}^{N} \phi_{2}\left(x_{\alpha}\right) \phi_{2}\left(x_{\alpha}\right) & & \\
&  & \ddots &  \\
&  &  & \sum_{\alpha=1}^{N} \phi_{n}\left(x_{\alpha}\right) \phi_{n}\left(x_{\alpha}\right)
\end{pmatrix}
\begin{pmatrix}
c_1\\ 
c_2\\ 
\vdots\\ 
c_n
\end{pmatrix}
=
\begin{pmatrix}
\sum_{\alpha=1}^{N} \phi_{1}\left(x_{\alpha}\right) y_{\alpha} \\
\sum_{\alpha=1}^{N} \phi_{2}\left(x_{\alpha}\right) y_{\alpha} \\
\vdots \\
\sum_{\alpha=1}^{N} \phi_{n}\left(x_{\alpha}\right) y_{\alpha}
\end{pmatrix}
$$

つまりは各成分 $c_i$ は以下の式から容易に計算できる．

$$
c_i = \frac{\sum_{\alpha=1}^{N} \phi_{i} \left(x_{\alpha}\right) y_{\alpha}}{\sum_{\alpha=1}^{N}  \phi_{i}\left(x_{\alpha}\right)^2 }, \qquad i=1,...,n
$$

### ベクトルの直交
正規方程式は自身以外との積が $0$ となるように関数を設計することで簡単に解くことができたが，近似対象が$N$ 個の離散データ $\{(x_i,y_i)\}_{i=1}^N$ ではなくベクトルや関数そのものであったときどのように最小二乗法や直交性を考えるべきかをここで紹介する．

そもそも **直交する** という表現はこれまでベクトルで登場したかと思う．具体的には，$m$次元のベクトル $\mathbf{a}, \mathbf{b}$ の内積が $0$ であるときこれらのベクトルは直交していると習ったはずだ．

$$
\left (\mathbf{a}, \mathbf{b} \right ) = \sum_{i=1}^m a_i b_i = a_1 b_1 + \cdots + a_m b_m = 0
$$

そして，ベクトルが直交するとは二つのベクトルが成す角が$90°$であった．このベクトルの直交性は，ベクトルを最小二乗法で近似する場合に直接利用することもできる．

以下にベクトルの最小二乗法についてまとめた．クリックすると確認できる．

`````{admonition} ベクトルの最小二乗法と直交性
:class: tip, dropdown

先ほどの離散データにおける最小二乗法と同様に，$m$ 次元列ベクトル $\mathbf{a}$ を $n$ 本の任意の￼$m$ 次元ベクトル $\{ \mathbf{u}_i \}^n_{i=1}$ の線形結合で表現する最小二乗法を考える．このときのベクトルの線型結合は

$$
\begin{align}
\mathbf{a} &\approx \sum_{k=1}^{n} c_{k} \mathbf{u}_k \\
&=c_1 \mathbf{u}_1 + c_2 \mathbf{u}_2 + \cdots c_n \mathbf{u}_n
\end{align}
$$

として表され，最小二乗法としてすべての成分を均一に近似するように最小化問題を考えることができる．

$$
J = \frac{1}{2} \left \| \mathbf{a} - \sum^{n}_{k=1} c_k \mathbf{u}_k \right \|^2 \rightarrow min
$$

同じく，関数 $J$ についてその偏導関数が $0$ となるように係数 $\{ c_i \}^n_{i=1}$ を求める．偏導関数は以下であり，

$$
\frac{\partial J}{\partial c_i} = -\left ( \mathbf{a}, \mathbf{u}_i \right ) + \sum_{k=1}^{n} c_k \left (\mathbf{u}_k, \mathbf{u}_i \right )
$$

正規方程式は以下となる．

$$
\begin{pmatrix}
\left \| \mathbf{u}_1\right \| &
\left ( \mathbf{u}_1, \mathbf{u}_2 \right ) &
\cdots &
\left ( \mathbf{u}_1, \mathbf{u}_n \right ) \\
\left ( \mathbf{u}_2, \mathbf{u}_1 \right ) &
\left \| \mathbf{u}_2 \right \|  &
\cdots &
\left ( \mathbf{u}_2, \mathbf{u}_n \right ) \\
\vdots & \vdots & \ddots & \vdots \\
\left ( \mathbf{u}_n, \mathbf{u}_1 \right ) &
\left ( \mathbf{u}_n, \mathbf{u}_2 \right ) &
\cdots &
\left \| \mathbf{u}_n \right \| 
\end{pmatrix}
\begin{pmatrix}
c_1\\ 
c_2\\ 
\vdots\\ 
c_n
\end{pmatrix}
=
\begin{pmatrix}
\left ( \mathbf{a}, \mathbf{u}_1 \right ) \\
\left ( \mathbf{a}, \mathbf{u}_2 \right ) \\
\vdots \\
\left ( \mathbf{a}, \mathbf{u}_n \right )
\end{pmatrix}
$$

ここでも自身との内積が $0$ となるような列ベクトルの集合 $\{ \mathbf{u}_i \}^n_{i=1}$ であると，以下のように正規方程式の非対角成分は $0$ となり，

$$
\begin{pmatrix}
\left \| \mathbf{u}_1\right \| &
&
&
\\
&
\left \| \mathbf{u}_2 \right \|  &
&
\\
&& \ddots & \\
&
&
&
\left \| \mathbf{u}_n \right \| 
\end{pmatrix}
\begin{pmatrix}
c_1\\ 
c_2\\ 
\vdots\\ 
c_n
\end{pmatrix}
=
\begin{pmatrix}
\left ( \mathbf{a}, \mathbf{u}_1 \right ) \\
\left ( \mathbf{a}, \mathbf{u}_2 \right ) \\
\vdots \\
\left ( \mathbf{a}, \mathbf{u}_n \right )
\end{pmatrix}
$$

各係数 $c_i$ は簡単に計算できる．

$$
c_i = \frac{\left (\mathbf{a}, \mathbf{u}_i \right )}{\left \| \mathbf{u}_i \right \|^ 2}, \qquad i=1,\ldots n
$$

ここからさらに，$\mathbf{u}$ を直交かつ単位ベクトルとなるように設計することを考える．

$$
\left \| \mathbf{u}_i \right \| = 1, \qquad i=1,\ldots,n
$$

このように $\{ \mathbf{u}_i \}^n_{i=1}$ のすべての要素が単位ベクトルであるとき **正規直交系** と呼び，$c_i$の解はさらにシンプルになる．

$$
c_i = \left (\mathbf{a}, \mathbf{u}_i \right ), \qquad i=1,\ldots n
$$

```{admonition} ベクトルのノルム
ベクトルのノルムは内積を用いて以下で定義される．

$$
\left \| \mathbf{a} \right \| = \sqrt{\left ( \mathbf{a}, \mathbf{a} \right )} = \sqrt{\sum^{m}_{i=1} a_i^2}
$$
```

```{admonition} 偏導関数の導出
$\partial J / \partial c_i$の式変形についてまとめたので参考にされたい．

$$
\begin{align}
\frac{\partial J}{\partial c_i} &= \frac{\partial}{\partial c_i} \left \{ \frac{1}{2} \left \| \mathbf{a} - \sum^{n}_{k=1} c_k \mathbf{u}_k \right \|^2 \right \} \\
&=\frac{\partial}{\partial c_i} \left \{ \frac{1}{2} \left ( \mathbf{a} - \sum^{n}_{k=1} c_k \mathbf{u}_k, \mathbf{a} - \sum^{n}_{l=1} c_l \mathbf{u}_l \right ) \right \} \\
&=\frac{\partial}{\partial c_i} \left \{ \frac{1}{2} \left ( \left ( \mathbf{a}, \mathbf{a} \right ) - 2 \left ( \mathbf{a}, \sum^{n}_{k=1} c_k \mathbf{u}_k  \right ) + \left ( \sum^{n}_{k=1} c_k \mathbf{u}_k, \sum^{n}_{l=1} c_l \mathbf{u}_l \right )\right ) \right \} \\
&=\frac{\partial}{\partial c_i} \left \{ \frac{1}{2} \left (\left \| \mathbf{a} \right \| ^2 - 2 \sum^{n}_{k=1} c_k \left ( \mathbf{a}, \mathbf{u}_k  \right ) + \sum^{n}_{k=1} \sum^{n}_{l=1} c_k c_l \left ( \mathbf{u}_k, \mathbf{u}_l \right )\right ) \right \} \\
&=-\left ( \mathbf{a}, \mathbf{u}_i \right ) + \sum_{k=1}^{n} c_k \left (\mathbf{u}_k, \mathbf{u}_i \right )
\end{align}
$$
```

`````

### 関数の直交
離散データとベクトルで直交を定義し，正規方程式からの最小二乗法を導出したのと同様に関数に対しても直交を定義することができる．

区間$[a,b]$で関数$f(x), g(x)$が以下のとき関数が直交するという．

$$
\int_{a}^{b}\ f(x) g(x) dx= 0
$$

そして，関数の集合 $\{\phi_i(x)\}^n_{i=0}$ が互いに直交するとき，区間 $[a,b]$ 上の直交関数系であるという．

$$
\int_{a}^{b}\ \phi_{i}\left(x\right) \phi_{j}\left(x\right) dx= 0, \qquad i \neq j
$$

この関数の直交性を利用してベクトルと同様に関数の最小二乗法を考えることもできる．

以下に関数の最小二乗法についてまとめた．クリックすると確認できる．

`````{admonition} 関数の最小二乗法と直交性
:class: tip, dropdown

区間 $[a,b]$ において，任意の関数 $f(x)$ を￼$n$ 個の関数 $\{ \phi_i(x) \}^n_{i=1}$ の線形結合

$$
\begin{align}
f(x) &\approx \sum_{k=1}^{n} c_{k} \phi_{k}(x)\\
&=c_1 \phi_{1}(x)+c_2 \phi_{2}(x) + \cdots c_n \phi_n(x)
\end{align}
$$

で表現する．ベクトルの場合と同様に，区間 $[a,b]$ 上のすべての点 $x$ で均一に近似するように次の最小化問題を考える．

$$
J = \frac{1}{2} \int_{a}^{b} \left ( f \left ( x \right ) - \sum_{n}^{k=1} c_{k} \phi_{k}\left ( x \right ) \right ) ^ 2 dx \rightarrow min
$$

同じく，関数 $J$ についてその偏導関数が $0$ となるように係数 $\{ c_i \}^n_{i=1}$ を求める．偏導関数は以下であり，

$$
\begin{align}
\frac{\partial J}{\partial c_i} &= -\int_{a}^{b} \left(f \left ( x \right ) - \sum_{k=1}^{n} c_{k} \phi_{k} \left(x \right) \right)  \phi_{i} \left( x \right) dx \\
&= \sum_{k=1}^{n} c_k \int_{a}^{b} \phi_k \left ( x \right ) \phi_i \left ( x \right ) dx 
- \int_{a}^{b} f \left ( x \right ) \phi_i \left ( x \right ) dx\\
\end{align}
$$

正規方程式は以下のように定まる．

$$
\begin{pmatrix}
\int_{a}^{b} \phi_{1} \left ( x \right ) ^2 dx &
\int_{a}^{b} \phi_{1} \left ( x \right ) \phi_{2} \left ( x \right ) dx &
\cdots &
\int_{a}^{b} \phi_{1} \left ( x \right ) \phi_{n} \left ( x \right ) dx \\
\int_{a}^{b} \phi_{2} \left ( x \right ) \phi_{1} \left ( x \right ) dx &
\int_{a}^{b} \phi_{2} \left ( x \right ) ^2 dx &
\cdots &
\int_{a}^{b} \phi_{2} \left ( x \right ) \phi_{n} \left ( x \right ) dx \\
\vdots & \vdots & \ddots & \vdots \\
\int_{a}^{b} \phi_{n} \left ( x \right ) \phi_{1} \left ( x \right ) dx &
\int_{a}^{b} \phi_{n} \left ( x \right ) \phi_{2} \left ( x \right ) dx &
\cdots &
\int_{a}^{b} \phi_{n} \left ( x \right ) ^2 dx
\end{pmatrix}
\begin{pmatrix}
c_1\\ 
c_2\\ 
\vdots\\ 
c_n
\end{pmatrix}
=
\begin{pmatrix}
\int_{a}^{b} \phi_{1} \left ( x \right ) f \left ( x \right ) dx \\
\int_{a}^{b} \phi_{2} \left ( x \right ) f \left ( x \right ) dx \\
\vdots \\
\int_{a}^{b} \phi_{n} \left ( x \right ) f \left ( x \right ) dx
\end{pmatrix}
$$

前述した関数の直交性より直交関数系 $\{ \phi_{i}(x) \}^{n}_{i=1}$ を設計すると，関数の場合も，正規方程式の非対角成分は $0$ となり，

$$
\begin{pmatrix}
\int_{a}^{b} \phi_{1} \left ( x \right )^2 dx &
&
&
\\
&
\int_{a}^{b} \phi_{2} \left ( x \right )^2 dx &
&
\\
&& \ddots &\\
&
&
&
\int_{a}^{b} \phi_{n} \left ( x \right )^2 dx
\end{pmatrix}
\begin{pmatrix}
c_1\\ 
c_2\\ 
\vdots\\ 
c_n
\end{pmatrix}
=
\begin{pmatrix}
\int_{a}^{b} \phi_{1} \left ( x \right ) f \left ( x \right ) dx \\
\int_{a}^{b} \phi_{2} \left ( x \right ) f \left ( x \right ) dx \\
\vdots \\
\int_{a}^{b} \phi_{n} \left ( x \right ) f \left ( x \right ) dx
\end{pmatrix}
$$

各係数は簡単に計算できる．

$$
c_i = \frac{\int_{a}^{b} \phi_{i} \left ( x \right ) f \left ( x \right) dx }{\int_{a}^{b} \phi_{i} \left ( x \right )^2 dx }, \qquad i=1,...,n
$$

```{admonition} 偏導関数の導出
$\partial J / \partial c_i$の式変形についてまとめたので参考にされたい．

$$
\begin{align}
\frac{\partial J}{\partial c_i} &= -\int_{a}^{b} \left(f \left ( x \right ) - \sum_{k=1}^{n} c_{k} \phi_{k} \left(x \right) \right)  \phi_{i} \left( x \right) dx \\
&=-\int_{a}^{b} f \left ( x \right ) \phi_{i} \left( x \right) dx + \int_{a}^{b} \sum_{k=1}^{n} c_{k} \phi_{k} \left(x \right)   \phi_{i} \left( x \right) dx \\
&= \sum_{k=1}^{n} \int_{a}^{b} c_k \phi_k \left ( x \right ) \phi_i \left ( x \right ) dx 
- \int_{a}^{b} f \left ( x \right ) \phi_i \left ( x \right ) dx\\
&= \sum_{k=1}^{n} c_k \int_{a}^{b} \phi_k \left ( x \right ) \phi_i \left ( x \right ) dx 
- \int_{a}^{b} f \left ( x \right ) \phi_i \left ( x \right ) dx\\
\end{align}
$$
```
`````

## 内積空間
ここまで離散データ，ベクトル，関数と直交性（自身以外との内積が$0$）について説明してきたが，そもそもベクトル以外の対象に内積を行うことに対して不自然さを感じた方もいるかと思う．ここでは内積演算が満たすべき条件（**内積の公理**）を紹介し，ベクトルだけでなく離散データや関数が内積の公理を満たした内積演算をしていることを確認する．

線形空間の元 $\mathbf{u}, \mathbf{v} \in \mathcal{L}$ について以下を満たすとき内積 $(\mathbf{u}, \mathbf{v})$ が定義される．

- 内積の正値性

    $$
    (\mathbf{u}, \mathbf{u}) \geq 0
    $$

- 内積の対称性

    $$
    (\mathbf{u}, \mathbf{v}) = (\mathbf{v}, \mathbf{u})
    $$

- 内積の線形性

    $$
    (c_1 \mathbf{u}_1 + c_2 \mathbf{u}_2, \mathbf{v}) = c_1 (\mathbf{u}, \mathbf{v}) + c_2 (\mathbf{u}, \mathbf{v})
    $$

上記の3つの条件（内積の公理）を満たす演算は **内積** となる．

加えて，内積が定義できるとノルムも定まる．

$$
\left \| \mathbf{u} \right \| = \sqrt{\left ( \mathbf{u}, \mathbf{u} \right )}
$$

内積とノルムが定義された線形空間を計量空間 $\mathcal{L}$ と呼ぶ．計量空間の任意の元 $\mathbf{u}, \mathbf{v} \in \mathcal{L}$ の内積が $0$ となるとき（$(\mathbf{u}, \mathbf{v}) = 0$），これらの元 は **直交する** という 



### 課題
**課題1:** 最小二乗法の説明において，$N$ 個の離散データ $\{x_\alpha \}^N_{\alpha=1}$ が与えられ，関数 $f(x), g(x)$ に対して，内積とノルムは次のように定義されていた．

内積：

$$
\left (f,g \right ) = \sum_{\alpha=1}^N f \left(x_\alpha \right) g \left(x_\alpha \right)
$$

ノルム：

$$
\left \| f \right \| = \sqrt{\left (f,f\right )} = \sqrt{\sum^{N}_{\alpha=1} f \left (x_\alpha\right )^2}
$$

この演算が内積の公理を満たすことを確かめよ．

**課題2:** $n$ 次元列ベクトル $\mathbf{a}, \mathbf{b}$ の内積とノルムは次のように定義されていた．

内積：

$$
\left (\mathbf{a}, \mathbf{b} \right ) = \sum_{i=1}^n a_i b_i
$$

ノルム：

$$
\left \| \mathbf{a} \right \| = \sqrt{\left ( \mathbf{a}, \mathbf{a} \right )} = \sqrt{\sum^{n}_{i=1} a_i^2}
$$

この演算が内積の公理を満たすことを確かめよ．

**課題３:** 区間 $[a,b]$ 上の連続関数 $f(x), g(x)$ に対して内積とノルムは次のように定義されていた．

内積：

$$
\left ( f, g \right ) = \int_{a}^{b} f\left(x\right) g\left(x\right) dx
$$

ノルム：

$$
\left \| \mathbf{f} \right \| = \sqrt{\left ( f, f  \right )} = \sqrt{ \int_{a}^{b} f\left(x\right) ^2 dx}
$$

この演算が内積の公理を満たすことを確かめよ．

## 次回の講義について
ここまで離散データ，ベクトル，関数と最小二乗法の近似対象の一般化を行った．その理由としては次回以降の講義で紹介するがフーリエ級数展開で登場する $\sin$関数と$\cos$関数は区間$[-\pi,\pi]$で直交する直交関数系であり，最小二乗法と密接に関係するからである．次回の講義ではこの性質を踏まえて，フーリエ級数展開を導出したい．